In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from urllib.parse import quote

In [5]:
# 웹 드라이버 설정
driver = webdriver.Chrome()
driver.maximize_window()


# 추출한 데이터 저장할 저장소 초기화
spot_list = []
name_list = []
review_list = [] 

# 무한 스크롤 함수
def infinite_loop(driver):
    # 최초 페이지 스크롤 설정
    # 스크롤 시키지 않았을 때의 전체 높이
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        # 윈도우 창을 0에서 위에서 설정한 전체 높이로 이동
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.0)
        # 스크롤 다운한 만큼의 높이를 신규 높이로 설정 
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        # 직전 페이지 높이와 신규 페이지 높이 비교
        if new_page_height == last_page_height:
            time.sleep(1.0)
            # 신규 페이지 높이가 이전과 동일하면, while문 break
            if new_page_height == driver.execute_script("return document.documentElement.scrollHeight"):
                break
        else:
            last_page_height = new_page_height

#--------------------------------------------------------------------------------------------------------------------
# 크롤링 코드 작성부

# @창 띄우고 관광지 이름으로 검색하는 코드@

# 본인 데이터 지정해서 spots 설정 변경해주세요
spots = ['서울둘레길1코스(수락산)','서울둘레길2코스(덕릉고개)','서울둘레길3코스(불암산)','서울둘레길4코스(망우용마산)','서울둘레길5코스(아차산)',
         '서울둘레길6코스(고덕산)','서울둘레길7코스(일자산)','서울둘레길8코스(장지탄천)','서울둘레길9코스(대모구룡산)','서울둘레길10코스(우면산)',
         '서울둘레길11코스(관악산)','서울둘레길12코스(호암산)','서울둘레길13코스(안양천상류)','서울둘레길14코스(안양천하류)','서울둘레길15코스(노을하늘공원)',
         '서울둘레길16코스(봉산앵봉산)','서울둘레길17코스(북한산은평)','서울둘레길18코스(북한산종로)','서울둘레길19코스(북한산성북)','서울둘레길20코스(북한산강북)',
         '서울둘레길21코스(북한산도봉)']

for j in range(len(spots)):
    spot = spots[j]
    # 특수 문자 포함 관광지명 인코딩해주는 코드
    encoded_spot = quote(spot)

    # 지도 페이지로 바로 들어가도록 변경
    # 기존에는 네이버 검색창에 검색 후 지도 페이지로 연결되도록 2 단계를 거침
    driver.get(f"https://m.map.naver.com/search2/search.naver?query={encoded_spot}")
    time.sleep(5)
    
#     #---------------------------------------------------------------------------------------------------------------------
    # 지도창으로 들어와서 관광지 이름 버튼 클릭하는 코드

    # map_paths : 지도 검색 페이지에서 해당 관광지의 XPATH 모음 리스트
    map_paths = [
        '//*[@id="ct"]/div[2]/ul/li[1]/div[1]/a/div/strong'
    ]

    # map XPATH를 통해 지도창으로 들어가는 버튼 클릭
    for map_path in map_paths:
        place_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, map_path))
                )

        place_button.click()

        time.sleep(2)

    # 해당 관광지 창으로 들어와서 리뷰버튼 클릭하는 코드
    review_xpaths = [
        '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[2]/span',
        '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[3]/span',
        '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[4]/span',
        '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[5]/span'
    ]

    timeout = True
    i = 0
    review_found = False # 리뷰 버튼이 클릭되면 나머지 review_xpath는 클릭 생략하도록 설정
    for review_xpath in review_xpaths:
        if review_found:
            break  # 리뷰를 찾았으면 루프를 종료

        try:
            review_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, review_xpath))
            )
            review_button.click()
            review_found = True

        except TimeoutException:
            if timeout:
                print('리뷰버튼 못참음')
                # 지도가 리뷰버튼 보다 앞에 존재하는 경우 리뷰버튼으로 넘어가지 못하는 오류 방지
                timeout = False
                continue
            
    
    # 무한 스크롤 함수
    infinite_loop(driver)
    
    # 리뷰 개수 범위 지정을 위한 요소 지정 >> 작성자명의 수
    review_people = driver.find_elements(By.CLASS_NAME, "P9EZi")
    print('리뷰 개수 :', len(review_people))
    
    # 리뷰 개수만큼 반복문 사용하여 작성자명, 별점 개수, 리뷰 내용 추출
    for i in range(len(review_people)):
        # 작성자명
        name = driver.find_elements(By.CLASS_NAME, "P9EZi")[i].text
        
        # 리뷰내용 >> 리뷰 없는 경우 존재하므로 있으면 추출하고 없으면 '리뷰 없음'으로 대체
        try:
            review = driver.find_elements(By.CLASS_NAME, "zPfVt")[i].text
        except:
            review = '리뷰 없음'
        
        # 결과 리스트에 담기
        spot_list.append(spot)
        name_list.append(name)
        review_list.append(review)
        
# 드라이버 종료
driver.quit()

리뷰 개수 : 1
리뷰 개수 : 2
리뷰 개수 : 2
리뷰 개수 : 3
리뷰 개수 : 9
리뷰 개수 : 5
리뷰 개수 : 2
리뷰 개수 : 2
리뷰 개수 : 1
리뷰 개수 : 0
리뷰 개수 : 5
리뷰 개수 : 3
리뷰 개수 : 1
리뷰 개수 : 1
리뷰 개수 : 1
리뷰 개수 : 5
리뷰 개수 : 0
리뷰 개수 : 0
리뷰 개수 : 0
리뷰 개수 : 2
리뷰 개수 : 4


In [7]:
# 결과 리스트로 dict 생성    
dict_df = {
    'spot' : spot_list,
    'name' : name_list,
    'review' : review_list,
}

# dict로 데이터 프레임 생성
df = pd.DataFrame(dict_df)
# 데이터 프레임 확인
df
# 데이터 프레임 저장
df.to_csv(f'../data/서울둘레길/네이버리뷰/네이버지도_리뷰모음.csv', index=False)

In [8]:
path = '../data/서울둘레길/네이버리뷰/네이버지도_리뷰모음.csv'

df = pd.read_csv(path)
df

,spot,name,review
0,서울둘레길1코스(수락산),shk****,"수락.불암산 서울 둘레길 1,2,3,4코스 완주\n했습니다.건강하게 완주하여 최고에..."
1,서울둘레길2코스(덕릉고개),노원쿨가이,서울둘레길2코스 난이도 상\n진짜 오르막 내리막의 끊임없는 반복\n그만큼 성취감도 큼
2,서울둘레길2코스(덕릉고개),dpaqmffkdtm,NaN
3,서울둘레길3코스(불암산),노원쿨가이,대중교통이편하고\n길도 편하고 자연이 좋은 서울둘레길3코스
4,서울둘레길3코스(불암산),꾸울1125,좋아요
5,서울둘레길4코스(망우용마산),핑크핑크아코,운동삼아 걸어가기 좋아요
6,서울둘레길4코스(망우용마산),핑크핑크아코,혼자 걷기 좋고\n초보들 다니기 좋아요
7,서울둘레길4코스(망우용마산),lalek6,서울둘레길 4코스 강남둘레길이라고도 부르네요
8,서울둘레길5코스(아차산),쮸공바라기,오랜만에 왔더니 아차산 산책로가 아주 잘 되있네요~~ 쭈욱 걷기만해도 산에 다녀온 ...
9,서울둘레길5코스(아차산),반장77,"서울 둘레길 중에서 제일 자주 가는 5코스 아차산길..\n시원한 조망, 특히 한강과..."
